In [20]:
import csv
import psycopg2
from psycopg2 import sql
import chardet
import pandas as pd  # Importando o Pandas

def detect_file_encoding(csv_file):
    """Detecta a codificação de um arquivo CSV usando chardet."""
    with open(csv_file, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(100000))  # Lê os primeiros 100KB
    return result['encoding']

def create_table_from_csv(csv_file, table_name, db_config):
    """Cria uma tabela no PostgreSQL a partir de um arquivo CSV e salva em Pandas."""
    # Detecta a codificação do arquivo CSV
    encoding = detect_file_encoding(csv_file)
    print(f"Codificação detectada: {encoding}")
    
    # Conecta ao PostgreSQL
    conn = None
    cur = None
    try:
        print("Conectando ao banco de dados PostgreSQL...")
        conn = psycopg2.connect(**db_config)
        cur = conn.cursor()

        # Lê o CSV em um DataFrame do Pandas e exibe a amostra
        df = pd.read_csv(csv_file, delimiter=';', encoding=encoding, on_bad_lines='skip')
        print("Amostra dos dados:\n", df.head())

        # Criação da tabela com tipos mapeados
        type_mapping = {
            'ChEMBL ID': 'TEXT',
            'Journal': 'TEXT',
            'Source ID': 'INTEGER',
            'Source Description': 'TEXT',
            'Title': 'TEXT',
            'PubMed ID': 'TEXT',
            'DOI': 'TEXT',
            'Patent ID': 'TEXT',
            'Authors': 'TEXT',
            'Year': 'INTEGER',
            'Volume': 'TEXT',
            'Issue': 'TEXT',
            'First Page': 'TEXT',
            'Last Page': 'TEXT',
            'Document Type': 'TEXT',
            'Abstract': 'TEXT',
            'Compounds': 'INTEGER',
            'Targets': 'INTEGER',
            'Activities': 'INTEGER',
            'ChEMBL Release': 'TEXT',
            'ChEMBL Release Creation Date': 'DATE'
        }

        # Gera comando SQL para criar a tabela
        create_table_query = sql.SQL("CREATE TABLE IF NOT EXISTS {} (\n").format(sql.Identifier(table_name))
        columns = [
            sql.SQL("{} {}").format(sql.Identifier(col), sql.SQL(type_mapping.get(col, 'TEXT')))
            for col in df.columns
        ]
        create_table_query += sql.SQL(",\n").join(columns)
        create_table_query += sql.SQL("\n);")

        # Executa comando para criar a tabela
        print("Executando comando SQL para criar a tabela...")
        cur.execute(create_table_query)
        conn.commit()
        print(f"Tabela {table_name} criada com sucesso!")

        # Inserção dos dados
        insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier(table_name),
            sql.SQL(', ').join(map(sql.Identifier, df.columns)),
            sql.SQL(', ').join(sql.Placeholder() * len(df.columns))
        )

        # Substitui strings vazias por None para colunas numéricas e insere os dados
        for _, row in df.iterrows():
            data = [
                None if pd.isna(cell) else cell for cell in row
            ]
            cur.execute(insert_query, data)

        conn.commit()
        print(f"Dados inseridos com sucesso na tabela {table_name}!")

        # Salva os dados em uma planilha Excel
        output_file = f"{table_name}.xlsx"
        df.to_excel(output_file, index=False)
        print(f"Dados salvos em {output_file}!")

    except (Exception, psycopg2.Error) as error:
        print("Erro ao conectar ao PostgreSQL ou criar a tabela:", error)
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()
        print("Conexão com o PostgreSQL fechada.")

# Configuração do banco de dados
db_config = {
    "dbname": "testando",
    "user": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "54321"
}

# Uso do script
csv_file = 'chagas.csv'
table_name = 'artigos_cientificos'

try:
    create_table_from_csv(csv_file, table_name, db_config)
except Exception as e:
    print(f"Ocorreu um erro ao executar o script: {e}")


Codificação detectada: utf-8
Conectando ao banco de dados PostgreSQL...
Amostra dos dados:
        ChEMBL ID                                    Journal  Source ID  \
0  CHEMBL1138713  Bioorganic & medicinal chemistry letters.          1   
1  CHEMBL1143452  Bioorganic & medicinal chemistry letters.          1   
2  CHEMBL1131724  Bioorganic & medicinal chemistry letters.          1   
3  CHEMBL1125964  Bioorganic & medicinal chemistry letters.          1   
4  CHEMBL1142328          Bioorganic & medicinal chemistry.          1   

      Source Description                                              Title  \
0  Scientific Literature  Synthesis and antitrypanosomal profile of new ...   
1  Scientific Literature  Synthesis and biological evaluation of 1-amino...   
2  Scientific Literature  The structure-based design and synthesis of se...   
3  Scientific Literature  Synthesis and biological activivity of juvenel...   
4  Scientific Literature  Imidazolidines as new anti-Trypanosoma cru